In [2]:
import os
import csv
import math
import itertools
import pickle

import pandas as pd
pd.set_option('display.max_columns', None)
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

from fairness_checker import *

In [3]:
# To load the model back
with open('./model/adult_RF.pkl', 'rb') as f:
    clf = pickle.load(f)
with open('./model/adult_synth.pkl', 'rb') as f:
    model = pickle.load(f)

In [4]:
class model_wrapper:
    def __init__(self, model):
        self.model = model

    def predict(self, file):
        df = pd.read_csv(file)
        X = self.preprocessing(df)

        return self.model.predict(X)

    def preprocessing(self, df):
        df.drop(labels=["workclass", "education", "occupation", "relationship", "race", "native.country"], axis = 1, inplace = True)
        df.drop(labels=["income"], axis = 1, inplace = True)

        return df

trained = model_wrapper(clf)

In [5]:
iter = 10

disparate_impact = []
demographic_parity = []
equalized_odds_1, equalized_odds_2 = [], []
equal_opportunity = []
accuracy_eqaulity = []
predictive_parity = []
equal_calibration = []
conditional_statistical_parity = []
predictive_equality = []
conditional_use_accuracy_equality_1, conditional_use_accuracy_equality_2 = [], []
positive_balance = []
negative_balance = []
mean_difference = []

for _ in range(iter):
    synth = model.synthetic_data(rows=30000)
    sdf = synth.df
    sdf.to_csv('./tmp/synth.csv', index=False)

    c = fairness_model_checker("./tmp/synth.csv", verbose=False)

    privileged_predicate = lambda row: row['sex'] != '0'
    positive_predicate = lambda Y: Y == 1
    truth_predicate = lambda row: row['income'] == '1'

    disparate_impact               .append( c.disparate_impact(0.8, trained, privileged_predicate, positive_predicate, value=True) )
    demographic_parity             .append( c.demographic_parity(0.2, trained, privileged_predicate, positive_predicate, value=True) )
    (x1, x2) = c.equalized_odds(0.2, trained, privileged_predicate, positive_predicate, truth_predicate, value=True)
    equalized_odds_1 .append( x1 )
    equalized_odds_2 .append( x2 )
    equal_opportunity              .append( c.equal_opportunity(0.2, trained, privileged_predicate, positive_predicate, truth_predicate, value=True) )
    accuracy_eqaulity              .append( c.accuracy_eqaulity(0.2, trained, privileged_predicate, positive_predicate, truth_predicate, value=True) )
    predictive_parity              .append( c.predictive_parity(0.2, trained, privileged_predicate, positive_predicate, truth_predicate, value=True) )
    # equal_calibration (n/a)
    conditional_statistical_parity .append( c.conditional_statistical_parity(0.2, trained, privileged_predicate, positive_predicate, lambda x: (lambda row: row['race'] == x), ('4',), value=True) )
    predictive_equality            .append( c.predictive_equality(0.2, trained, privileged_predicate, positive_predicate, truth_predicate, value=True) )
    (y1, y2) = c.conditional_use_accuracy_equality(0.2, trained, privileged_predicate, positive_predicate, truth_predicate, value=True)
    conditional_use_accuracy_equality_1 .append( y1 )
    conditional_use_accuracy_equality_2 .append( y2 )
    # positive_balance (n/a)
    # negative_balance (n/a)
    mean_difference .append( c.mean_difference(0.2,trained, privileged_predicate, positive_predicate, value=True) )

/usr/local/anaconda3/envs/mbi/lib/python3.13/site-packages/sklearn/base.py:486: UserWarning: X has feature names, but RandomForestClassifier was fitted without feature names
  warnings.warn(
/usr/local/anaconda3/envs/mbi/lib/python3.13/site-packages/sklearn/base.py:486: UserWarning: X has feature names, but RandomForestClassifier was fitted without feature names
  warnings.warn(
/usr/local/anaconda3/envs/mbi/lib/python3.13/site-packages/sklearn/base.py:486: UserWarning: X has feature names, but RandomForestClassifier was fitted without feature names
  warnings.warn(
/usr/local/anaconda3/envs/mbi/lib/python3.13/site-packages/sklearn/base.py:486: UserWarning: X has feature names, but RandomForestClassifier was fitted without feature names
  warnings.warn(
/usr/local/anaconda3/envs/mbi/lib/python3.13/site-packages/sklearn/base.py:486: UserWarning: X has feature names, but RandomForestClassifier was fitted without feature names
  warnings.warn(
/usr/local/anaconda3/envs/mbi/lib/python3.13/

In [6]:
og_disparate_impact = 0
og_demographic_parity = 0
og_equalized_odds_1, og_equalized_odds_2 = 0, 0
og_equal_opportunity = 0
og_accuracy_eqaulity = 0
og_predictive_parity = 0
og_equal_calibration = 0
og_conditional_statistical_parity = 0
og_predictive_equality = 0
og_conditional_use_accuracy_equality_1, og_conditional_use_accuracy_equality_2 = 0, 0
og_positive_balance = 0
og_negative_balance = 0
og_mean_difference = 0

cog = fairness_model_checker("./data/adult_processed.csv", verbose=False)

privileged_predicate = lambda row: row['sex'] == '0'
positive_predicate = lambda Y: Y == 1
truth_predicate = lambda row: row['income'] == '1'

og_disparate_impact = cog.disparate_impact(0.8, trained, privileged_predicate, positive_predicate, value=True)
og_demographic_parity = cog.demographic_parity(0.2, trained, privileged_predicate, positive_predicate, value=True)
(og_equalized_odds_1, og_equalized_odds_2) = cog.equalized_odds(0.2, trained, privileged_predicate, positive_predicate, truth_predicate, value=True)
og_equal_opportunity = cog.equal_opportunity(0.2, trained, privileged_predicate, positive_predicate, truth_predicate, value=True)
og_accuracy_eqaulity = cog.accuracy_eqaulity(0.2, trained, privileged_predicate, positive_predicate, truth_predicate, value=True)
og_predictive_parity = cog.predictive_parity(0.2, trained, privileged_predicate, positive_predicate, truth_predicate, value=True)
# equal_calibration (n/a)
og_conditional_statistical_parity = cog.conditional_statistical_parity(0.2, trained, privileged_predicate, positive_predicate, lambda x: (lambda row: row['race'] == x), ('4',), value=True)
og_predictive_equality = cog.predictive_equality(0.2, trained, privileged_predicate, positive_predicate, truth_predicate, value=True)
(og_conditional_use_accuracy_equality_1, og_conditional_use_accuracy_equality_2) = cog.conditional_use_accuracy_equality(0.2, trained, privileged_predicate, positive_predicate, truth_predicate, value=True)
# positive_balance (n/a)
# negative_balance (n/a)
og_mean_difference = cog.mean_difference(0.2, trained, privileged_predicate, positive_predicate, value=True)

/usr/local/anaconda3/envs/mbi/lib/python3.13/site-packages/sklearn/base.py:486: UserWarning: X has feature names, but RandomForestClassifier was fitted without feature names
  warnings.warn(
/usr/local/anaconda3/envs/mbi/lib/python3.13/site-packages/sklearn/base.py:486: UserWarning: X has feature names, but RandomForestClassifier was fitted without feature names
  warnings.warn(
/usr/local/anaconda3/envs/mbi/lib/python3.13/site-packages/sklearn/base.py:486: UserWarning: X has feature names, but RandomForestClassifier was fitted without feature names
  warnings.warn(
/usr/local/anaconda3/envs/mbi/lib/python3.13/site-packages/sklearn/base.py:486: UserWarning: X has feature names, but RandomForestClassifier was fitted without feature names
  warnings.warn(
/usr/local/anaconda3/envs/mbi/lib/python3.13/site-packages/sklearn/base.py:486: UserWarning: X has feature names, but RandomForestClassifier was fitted without feature names
  warnings.warn(
/usr/local/anaconda3/envs/mbi/lib/python3.13/

In [7]:
print("name               ", "og          ", "synth       ", "diff        ", "std")
print("demographic_parity ", f"{og_demographic_parity:.10f}", f"{np.mean(np.array(demographic_parity)):.10f}", f"{abs(og_demographic_parity - np.mean(np.array(demographic_parity))):.10f}", np.std(np.array(demographic_parity)))
print("accuracy_eqaulity  ", f"{og_accuracy_eqaulity:.10f}", f"{np.mean(np.array(accuracy_eqaulity)):.10f}", f"{abs(og_accuracy_eqaulity - np.mean(np.array(accuracy_eqaulity))):.10f}", np.std(np.array(accuracy_eqaulity)))
print("equalized_odds_1   ", f"{og_equalized_odds_1:.10f}", f"{np.mean(np.array(equalized_odds_1)):.10f}", f"{abs(og_equalized_odds_1 - np.mean(np.array(equalized_odds_1))):.10f}", np.std(np.array(equalized_odds_1)))
print("equalized_odds_2   ", f"{og_equalized_odds_2:.10f}", f"{np.mean(np.array(equalized_odds_2)):.10f}", f"{abs(og_equalized_odds_2 - np.mean(np.array(equalized_odds_2))):.10f}", np.std(np.array(equalized_odds_2)))
print("accuracy_equality_1", f"{og_conditional_use_accuracy_equality_1:.10f}", f"{np.mean(np.array(conditional_use_accuracy_equality_1)):.10f}", f"{abs(og_conditional_use_accuracy_equality_1 - np.mean(np.array(conditional_use_accuracy_equality_1))):.10f}", np.std(np.array(conditional_use_accuracy_equality_1)))
print("accuracy_equality_2", f"{og_conditional_use_accuracy_equality_2:.10f}", f"{np.mean(np.array(conditional_use_accuracy_equality_2)):.10f}", f"{abs(og_conditional_use_accuracy_equality_2 - np.mean(np.array(conditional_use_accuracy_equality_2))):.10f}", np.std(np.array(conditional_use_accuracy_equality_2)))
print("mean_difference    ", f"{og_mean_difference:.10f}", f"{np.mean(np.array(mean_difference)):.10f}", f"{abs(og_mean_difference - np.mean(np.array(mean_difference))):.10f}", np.std(np.array(mean_difference)))

print(
abs(og_demographic_parity - np.mean(np.array(demographic_parity)))+
abs(og_accuracy_eqaulity - np.mean(np.array(accuracy_eqaulity)))+
abs(og_equalized_odds_1 - np.mean(np.array(equalized_odds_1)))+
abs(og_equalized_odds_2 - np.mean(np.array(equalized_odds_2)))+
abs(og_conditional_use_accuracy_equality_1 - np.mean(np.array(conditional_use_accuracy_equality_1)))+
abs(og_conditional_use_accuracy_equality_2 - np.mean(np.array(conditional_use_accuracy_equality_2)))+
abs(og_mean_difference - np.mean(np.array(mean_difference)))
)

name                og           synth        diff         std
demographic_parity  0.1933447448 0.1056558354 0.0876889093 0.0030916137906068174
accuracy_eqaulity   0.0250249201 0.1387226036 0.1136976835 0.003887801014105579
equalized_odds_1    0.0175621188 0.0850076190 0.0674455002 0.003022056671385655
equalized_odds_2    0.0114542011 0.0590448595 0.0475906583 0.014947525495322567
accuracy_equality_1 0.1941115666 0.1436322323 0.0504793344 0.024634493941687395
accuracy_equality_2 0.1102330179 0.1443790644 0.0341460466 0.006213490989464394
mean_difference     0.1801306340 0.1027585890 0.0773720450 0.002029917439877628
0.47842017725511077
